# Cleaning up data

### Data Source
The raw data text file contains a detailed summary of the land-surface average results produced by the Berkeley Averaging method. This analysis was run on 12-Mar-2019 08:22:40.
Temperatures are in Celsius and reported as anomalies relative to the Jan 1951-Dec 1980 average (8.65 Celsius)
Uncertainties represent the 95% confidence interval for statistical and spatial undersampling effects. 

Source url: http://berkeleyearth.lbl.gov/auto/Global/Complete_TAVG_complete.txt

### Importing neccessary libraries

In [1]:
import pandas as pd
import types
from ibm_botocore.client import Config
import ibm_boto3
import io
import numpy as np
from datetime import datetime,timedelta

### Getting data from object storage into streamingbody object
We downloaded the data into the data-assest. But one can directly download the datafile from the weblink provided above.

In [2]:
#!wget http://berkeleyearth.lbl.gov/auto/Global/Complete_TAVG_complete.txt

--2019-03-26 02:39:45--  http://berkeleyearth.lbl.gov/auto/Global/Complete_TAVG_complete.txt
Resolving berkeleyearth.lbl.gov (berkeleyearth.lbl.gov)... 128.3.29.26
Connecting to berkeleyearth.lbl.gov (berkeleyearth.lbl.gov)|128.3.29.26|:80... connected.
HTTP request sent, awaiting response... 200 OK
Length: 318233 (311K) [text/plain]
Saving to: ‘Complete_TAVG_complete.txt’

100%[======================================>] 318,233      812KB/s   in 0.4s   

2019-03-26 02:39:46 (812 KB/s) - ‘Complete_TAVG_complete.txt’ saved [318233/318233]



In [ ]:
# @hidden_cell
# The following code contains the credentials for a file in your IBM Cloud Object Storage.
# You might want to remove those credentials before you share your notebook.
credentials_1 = {
    'IBM_API_KEY_ID': '',
    'IAM_SERVICE_ID': '',
    'ENDPOINT': '',
    'IBM_AUTH_ENDPOINT': '',
    'BUCKET': '',
    'FILE': 'GlobalTemperature.txt'
}
cos = ibm_boto3.client('s3',
                       ibm_api_key_id=credentials_1['IBM_API_KEY_ID'],
                       ibm_service_instance_id=credentials_1['IAM_SERVICE_ID'],
                       ibm_auth_endpoint=credentials_1['IBM_AUTH_ENDPOINT'],
                       config=Config(signature_version='oauth'),
                       endpoint_url=credentials_1['ENDPOINT'])
fileobject = cos.get_object(Bucket=credentials_1['BUCKET'], Key=credentials_1['FILE'])['Body'] # create the streamingbody object

### Step 1. Streamingbody object to list of strings

1. convert the input data into string format. 
2. Split the strings of sentences into lists by delimiter of end of sentence (\\n).
3. Remove the first 32 sentences that have the description about the dataset. We only need the datatable.

In [18]:
string_data=str(fileobject.read()) # read the fileobject and convert it to string
string_data=string_data.split('\\n') # split the string into seperate sentences
string_data=string_data[32:] # exclude sentences of initial introduction and only include datatable
string_data[:10]

['%                  Monthly          Annual          Five-year        Ten-year        Twenty-year',
 '% Year, Month,  Anomaly, Unc.,   Anomaly, Unc.,   Anomaly, Unc.,   Anomaly, Unc.,   Anomaly, Unc.',
 ' ',
 '  1750     1     0.302  5.003    -0.432  1.274       NaN    NaN       NaN    NaN       NaN    NaN',
 '  1750     2    -1.412  4.161    -0.352  1.181       NaN    NaN       NaN    NaN       NaN    NaN',
 '  1750     3    -0.268  2.682    -0.367  1.082       NaN    NaN       NaN    NaN       NaN    NaN',
 '  1750     4     0.702  1.966    -0.383  1.070       NaN    NaN       NaN    NaN       NaN    NaN',
 '  1750     5    -0.719  2.953    -0.719  0.946       NaN    NaN       NaN    NaN       NaN    NaN',
 '  1750     6    -0.399  3.504    -0.677  0.932       NaN    NaN       NaN    NaN       NaN    NaN',
 '  1750     7     0.469  2.692    -0.755  0.924       NaN    NaN       NaN    NaN       NaN    NaN']

### Step 2. Create DataFrame
From the list of strings create a dataframe with columns given in the raw text file.

In [20]:
Rows_list=[]
for string in string_data[3:-1]:
    Rows_list.append(string.split())
df=pd.DataFrame(Rows_list,columns=['year','month','Monthly_Anomaly','Monthly_Uncertainity',
                                'Annual_Anomaly','Annual_Uncertainity','Five-year_Anomaly','Five-year_Uncertainity',
                                'Ten-year_Anomaly','Ten-year_Uncertainity','Twenty-year_Anomaly','Twenty-year_Uncertainity'])
df.head()

,year,month,Monthly_Anomaly,Monthly_Uncertainity,Annual_Anomaly,Annual_Uncertainity,Five-year_Anomaly,Five-year_Uncertainity,Ten-year_Anomaly,Ten-year_Uncertainity,Twenty-year_Anomaly,Twenty-year_Uncertainity
0,1750,1,0.302,5.003,-0.432,1.274,NaN,NaN,NaN,NaN,NaN,NaN
1,1750,2,-1.412,4.161,-0.352,1.181,NaN,NaN,NaN,NaN,NaN,NaN
2,1750,3,-0.268,2.682,-0.367,1.082,NaN,NaN,NaN,NaN,NaN,NaN
3,1750,4,0.702,1.966,-0.383,1.070,NaN,NaN,NaN,NaN,NaN,NaN
4,1750,5,-0.719,2.953,-0.719,0.946,NaN,NaN,NaN,NaN,NaN,NaN


We are going to focus on the monthly average only so we get rid of rest of the columns. Also, we convert the data-entries to numeric format from string format.

In [21]:
# We only select the Monthly anomaly and uncertainity
df_filtered=df[['year','month','Monthly_Anomaly','Monthly_Uncertainity']]
for col in df_filtered.columns:
    df_filtered[col]=pd.to_numeric(df_filtered[col],errors='coerce')
df_filtered.head()

/usr/local/src/conda3_runtime/home/envs/DSX-Python35-Spark/lib/python3.5/site-packages/ipykernel/__main__.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy


,year,month,Monthly_Anomaly,Monthly_Uncertainity
0,1750,1,0.302,5.003
1,1750,2,-1.412,4.161
2,1750,3,-0.268,2.682
3,1750,4,0.702,1.966
4,1750,5,-0.719,2.953


### Step 3. Fix null values
Find the time-steps with NaN entries

In [22]:
# Let's check for NaN entries and fill zeros to them as we can assign the average of previous and next month
df_filtered[df_filtered['Monthly_Anomaly'].isnull()]

,year,month,Monthly_Anomaly,Monthly_Uncertainity
23,1751,12,NaN,NaN


Insert average value of previous time-step and next time-step in place of the NaN value.

In [23]:
NaN_idx=df_filtered[df_filtered['Monthly_Anomaly'].isnull()].index
df_filtered.loc[NaN_idx,'Monthly_Anomaly']=(list(df_filtered.loc[NaN_idx-1,'Monthly_Anomaly'])[0]+list(df_filtered.loc[(NaN_idx+1),'Monthly_Anomaly'])[0])/2
df_filtered.loc[NaN_idx,'Monthly_Uncertainity']=(list(df_filtered.loc[NaN_idx-1,'Monthly_Uncertainity'])[0]+list(df_filtered.loc[(NaN_idx+1),'Monthly_Uncertainity'])[0])/2

/usr/local/src/conda3_runtime/home/envs/DSX-Python35-Spark/lib/python3.5/site-packages/pandas/core/indexing.py:537: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self.obj[item] = s


### Step 4.Create new features
We create a new feature called 'Monthly_Temperature' from Monthly_Anomaly by adding the **estimated average Jan 1951-Dec 1980 absolute temperature (C): 8.65**.

In [24]:
# Create new column with actual average monthly temperature by adding
df_filtered['Monthly_Temperature']=df_filtered['Monthly_Anomaly']+8.65
df_filtered.head()

/usr/local/src/conda3_runtime/home/envs/DSX-Python35-Spark/lib/python3.5/site-packages/ipykernel/__main__.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  from ipykernel import kernelapp as app


,year,month,Monthly_Anomaly,Monthly_Uncertainity,Monthly_Temperature
0,1750,1,0.302,5.003,8.952
1,1750,2,-1.412,4.161,7.238
2,1750,3,-0.268,2.682,8.382
3,1750,4,0.702,1.966,9.352
4,1750,5,-0.719,2.953,7.931


We create new feature called 'Date' using the Year and month features. We save this feature in the datetime format.

In [25]:
df_date = pd.DataFrame({'year': list(df_filtered.year),
                       'month': list(df_filtered.month),
                       'day': list(np.repeat(1,len(df_filtered)))})
df_filtered['Date']=pd.to_datetime(df_date)

/usr/local/src/conda3_runtime/home/envs/DSX-Python35-Spark/lib/python3.5/site-packages/ipykernel/__main__.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy


Finally, we get rid of the 'year', 'month' and 'Monthly_Anomaly' columns.

In [26]:
df_filtered.drop(columns=['year','month','Monthly_Anomaly'],inplace=True)

/usr/local/src/conda3_runtime/home/envs/DSX-Python35-Spark/lib/python3.5/site-packages/ipykernel/__main__.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  if __name__ == '__main__':


In [27]:
df_filtered.head()

,Monthly_Uncertainity,Monthly_Temperature,Date
0,5.003,8.952,1750-01-01
1,4.161,7.238,1750-02-01
2,2.682,8.382,1750-03-01
3,1.966,9.352,1750-04-01
4,2.953,7.931,1750-05-01


### Save the cleaned up data
Congratulation all data cleaning is done, starting from the raw text file we got the dataframe with monthly average temperature and uncertainity. Let's save this dataframe

In [35]:
df_filtered.to_csv('Global_temperature.csv',index=False)
with open('Global_temperature.csv', 'rb') as data:
     cos.upload_fileobj(data,  credentials_1['BUCKET'], 'Global_temperature.csv')